## Import Libraries and data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\bulle\Documents\Python-Instancart Basket Analysis'
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [3]:
cust

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799


In [4]:
# change column names
cust.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age','n_dependants' : 'nb_dependants'}, inplace = True)


In [5]:
# Checking for missing values
cust.isnull().sum()

user_id              0
first_name       11259
last_name            0
gender               0
state                0
age                  0
date_joined          0
nb_dependants        0
fam_status           0
income               0
dtype: int64

In [6]:
# finding "NaN" missing value
cust['first_name'].value_counts(dropna = False) 


NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

Since 11259 are "NaN" in the first name column, It might be more accurate to use last name column for future reference.

In [7]:
# checking for duplicates
cust[cust.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,nb_dependants,fam_status,income


There are no duplicates in the cust dataframe.

In [8]:
# checking for abnormal data in columns
cust.describe()

,user_id,age,nb_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        206209 non-null  int64 
 1   first_name     194950 non-null  object
 2   last_name      206209 non-null  object
 3   gender         206209 non-null  object
 4   state          206209 non-null  object
 5   age            206209 non-null  int64 
 6   date_joined    206209 non-null  object
 7   nb_dependants  206209 non-null  int64 
 8   fam_status     206209 non-null  object
 9   income         206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


All columns seem logical.

In [10]:
# checking for mixed data type in columns
for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)


first_name


In [11]:
# converting first_name data type to string
cust['first_name'] = cust['first_name'].astype('str')
cust['user_id'] = cust['user_id'].astype('int')

In [12]:
cust['age'] = cust['age'].astype('int8')
cust['nb_dependants'] = cust['nb_dependants'].astype('int8')
cust['income'] = cust['income'].astype('float')

# Q6 Merging customer data with prepared instantcart data 

In [13]:
# Import orders_products_merged pkl
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_4-9.pkl'))


In [14]:
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('int')

In [15]:
ords_prods_merge['order_id'] = ords_prods_merge['order_id'].astype('int')

In [16]:
ords_prods_merge.dtypes

order_id                     int32
user_id                      int32
eval_set                    object
order_number                  int8
order_day_of_the_week         int8
order_hour_of_day             int8
days_since_prior_order     float16
product_id                   int32
add_to_cart_order             int8
reordered                     int8
_merge                    category
Unnamed: 0                   int64
product_name                object
aisle_id                     int32
department_id                int32
prices                     float32
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                     int8
loyalty_flag                object
spending_power             float32
Spender_flag                object
frequency                  float32
frequency_flag              object
dtype: object

In [18]:
ords_prods_custs = ords_prods_merge.merge(cust, on = 'user_id')

In [ ]:
ords_prods_custs.head(20)

In [19]:
# Export new merged file as pkl
ords_prods_custs.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_custs_p1.pkl'))

In [ ]:
ords_prods_custs.describe()